# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [ ]:
pip install pymorphy2

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [ ]:
from nltk.metrics.distance import edit_distance
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import random
import pandas as pd

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
words = []
with open('litw-win.txt', 'r', encoding='windows-1251') as f:
    for line in f:
        words.append(line.split()[1])
text = "с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий"
for token in word_tokenize(text, language="russian"):
    if token not in words:
        distances = [edit_distance(token, word) for word in words]
        closest_word = words[distances.index(min(distances))]
        print(closest_word, end=' ')
    else:
        print(token, end=' ')

с величайшим усилием выбравшись из потока убегающих людей кутузов со свитой уменьшившейся вдвое поехал на звуки выстрелов русских орудий 

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

In [ ]:
text = 'Считайте слова из файла litw-win.txt и запишите их в список words. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка words. Считайте, что в слове есть опечатка, если данное слово не содержится в списке words.'
stemmer = SnowballStemmer(language="russian")
morph = pymorphy2.MorphAnalyzer()
tokenized_text = word_tokenize(text, language="russian")
stemmed_words = [stemmer.stem(word) for word in tokenized_text]
normalized_words = [morph.parse(word)[0].normal_form for word in tokenized_text]
print(stemmed_words)
print(normalized_words)


['счита', 'слов', 'из', 'файл', 'litw-win.txt', 'и', 'запиш', 'их', 'в', 'список', 'words', '.', 'в', 'зада', 'предложен', 'исправьт', 'все', 'опечатк', ',', 'замен', 'слов', 'с', 'опечатк', 'на', 'ближайш', '(', 'в', 'смысл', 'расстоян', 'левенштейн', ')', 'к', 'ним', 'слов', 'из', 'списк', 'words', '.', 'счита', ',', 'что', 'в', 'слов', 'ест', 'опечатк', ',', 'есл', 'дан', 'слов', 'не', 'содерж', 'в', 'списк', 'words', '.']
['считать', 'слово', 'из', 'файл', 'litw-win.txt', 'и', 'записать', 'они', 'в', 'список', 'words', '.', 'в', 'задать', 'предложение', 'исправить', 'всё', 'опечатка', ',', 'заменить', 'слово', 'с', 'опечатка', 'на', 'близкий', '(', 'в', 'смысл', 'расстояние', 'левенштейн', ')', 'к', 'они', 'слово', 'из', 'список', 'words', '.', 'считать', ',', 'что', 'в', 'слово', 'есть', 'опечатка', ',', 'если', 'данный', 'слово', 'не', 'содержаться', 'в', 'список', 'words', '.']


3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

In [ ]:
text = 'Считайте слова из файла litw-win.txt и запишите их в список words. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка words. Считайте, что в слове есть опечатка, если данное слово не содержится в списке words.'
sentences = sent_tokenize(text, language="russian")
vectorizer = CountVectorizer()
vectorizer.fit(sentences)  # составляет словарь
vector = vectorizer.transform(sentences)
vector_array = vector.toarray()
vector_array

array([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 2, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1]])

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [ ]:
descriptions_df = pd.read_csv('preprocessed_descriptions.csv', index_col='Unnamed: 0')
list_descriptions = descriptions_df['preprocessed_descriptions'].to_list()
words = list(set(word for description in list_descriptions for word in word_tokenize(str(description))))
print(words)

['guestimatedi', '212416', 'togethersfrom', 'bolivia', 'dfby', 'good', 'wendys', 'battered', 'sweetker', 'device', 'at', 'spirit', 'temptations', 'spartanburg', 'traci', 'meatbut', 'drinkable', 'fancy', 'midnite', 'slicingserving', 'bellyfilling', '2009', 'chickens', 'overstuffed', 'xochimilco', 'enewsletter', 'malic', 'hooks', 'familycan', 'prepareahead', 'maw', 'byerlys', 'turnipmany', 'showcases', 'symbolizes', 'drizzling', 'fresco', 'fatsmashforumcom', 'heaventime', 'refirgerate', 'tonys', 'antonieta', 'parsnip', 'reddish', 'clintons', 'complements', 'goodwin', 'shoo', 'groups', 'evens', 'winemustard', 'traditionaly', '501', 'drinkfrom', 'him', 'collards', 'immensely', 'haunting', 'elevated', 'e3', 'jumper', 'johnston', 'pacific', 'awesomefor', 'smile', '19291955', 'headed', 'nutfree', 'sothe', 'finest', '1979', 'purnell', 'mlzer', 'waiter', 'hungers', 'sanyo', 'cookers', 'visitor', 'dilly', 'zimmerman', 'takealongs', 'wwwbhgcom', 'treece', 'creativebut', 'eagerly', 'incapable', 'c

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [ ]:
import random

examples = list(zip(random.sample(words, 5), random.sample(words, 5)))
for i,j in examples:
  print(i, j, '-->', edit_distance(i, j))

martha whoa --> 5
widerimmed coolingchilling --> 13
dianna susannah --> 4
whitby cellophane --> 10
3use recipie --> 6


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [ ]:
def find_nearest_words(word, k):
    global words
    word_distances = [(edit_distance(word, i), i) for i in words]
    sorted_distances = sorted(word_distances)[:k+1]
    return sorted_distances
print(find_nearest_words('cambodiansed', 3))


[(3, 'cambodian'), (5, 'canadians'), (5, 'carbonated'), (5, 'combined')]


### Стемминг, лемматизация

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [ ]:
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
sb = SnowballStemmer(language="english")
words = list(set(k for j in [word_tokenize(str(i)) for i in list_descriptions] for k in j))
table = pd.DataFrame({
    'word': words,
    'stemmed_word': [sb.stem(i) for i in words],
    'normalized_word': [wnl.lemmatize(i) for i in words]})
table.set_index('word')

,stemmed_word,normalized_word
word,,
guestimatedi,guestimatedi,guestimatedi
212416,212416,212416
togethersfrom,togethersfrom,togethersfrom
bolivia,bolivia,bolivia
dfby,dfbi,dfby
...,...,...
decorative,decor,decorative
moisten,moisten,moisten
incuded,incud,incuded


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [ ]:
 from nltk.corpus import stopwords 
 nltk.download('stopwords') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import collections

descript = pd.read_csv('preprocessed_descriptions.csv', index_col='Unnamed: 0')
list_descript = descript['preprocessed_descriptions'].tolist()
words = [k for j in [word_tokenize(str(i)) for i in list_descript] for k in j]
stop_words = set(stopwords.words('english'))
words_ = [word for word in words[:100] if word not in stop_words]
total_words = len(words)
filtered_words = len(words_)
stopwords_ratio = (total_words - filtered_words) / total_words
counter1 = collections.Counter(words)
counter2 = collections.Counter(words_)
print('Всего слов:', total_words)
print('Слов без стоп-слов:', filtered_words)
print('Доля стоп-слов:', stopwords_ratio)
print('Топ-5 слов до удаления:', counter1.most_common(5))
print('Топ-5 слов после удаления:', counter2.most_common(5))

Всего слов: 1069885
Слов без стоп-слов: 45
Доля стоп-слов: 0.9999579394047023
Топ-5 слов до удаления: [('the', 40072), ('a', 34951), ('and', 30245), ('this', 26859), ('i', 24836)]
Топ-5 слов после удаления: [('recipe', 2), ('restaurant', 2), ('good', 2), ('original', 1), ('created', 1)]


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

descript = pd.read_csv('preprocessed_descriptions.csv', index_col='Unnamed: 0')
samples = descript.sample(5)
five_samples = samples['preprocessed_descriptions'].tolist()
vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')
vectors = vectorizer.fit_transform(five_samples).toarray()
vectors[0]

array([0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.2773501,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.2773501,
       0.       , 0.2773501, 0.2773501, 0.       , 0.       , 0.2773501,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.2773501,
       0.       , 0.       , 0.2773501, 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.2773501, 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.2773501, 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.2773501,
       0.       , 0.       , 0.       , 0.       , 0.2773501, 0.2773501,
       0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
       0.       , 0.       , 0.       , 0.       , 

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial import distance

descript = pd.read_csv('preprocessed_descriptions.csv', index_col='Unnamed: 0')
samples = descript.sample(5)
five_samples = samples['preprocessed_descriptions'].tolist()
vectorizer = TfidfVectorizer(analyzer='word', stop_words='english')
vectors = vectorizer.fit_transform(five_samples).toarray()
names = samples['name'].tolist()
distances = [[distance.cosine(vectors[names.index(i)], vectors[j]) for j in range(5)] for i in names]
data = pd.DataFrame(distances, columns=names, index=names)
data

,popcorn with parmesan and pecorino,parmesan crusted lamb cutlets,raw spaghetti with marinara sauce,bread machine indian bread,garlic parmesan chicken wings
popcorn with parmesan and pecorino,0.000000,0.968495,1.000000,0.863597,0.987892
parmesan crusted lamb cutlets,0.968495,0.000000,1.000000,0.960199,0.989221
raw spaghetti with marinara sauce,1.000000,1.000000,0.000000,1.000000,0.961082
bread machine indian bread,0.863597,0.960199,1.000000,0.000000,0.953333
garlic parmesan chicken wings,0.987892,0.989221,0.961082,0.953333,0.000000
